In [2]:
import os
import pickle
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

import warnings
warnings.filterwarnings('ignore')

In [3]:
conv_data = pd.read_excel("data/complete_annotation_data.xlsx")
conv_data

,id,conversation_id,text,participant,human_int
0,36289,212,"student: sorry, it's my name as usual ) hi ) ...",6101885b0f3e7fdafc8fd5d1,0
1,36290,212,"student: fine, but quite (razbitaya) ) teache...",6101885b0f3e7fdafc8fd5d1,1
2,36291,212,student: i don't know if it's the weather chan...,6101885b0f3e7fdafc8fd5d1,3
3,36292,212,"student: i mean i'm ok, but i fell not 100% t...",6101885b0f3e7fdafc8fd5d1,1
4,36293,212,student: i see! it's so hard to be at home all...,6101885b0f3e7fdafc8fd5d1,2
...,...,...,...,...,...
19963,65706,166,"teacher: yes, historical is about history as a...",60317fc2c8f6320ecde4bcb3,1
19964,65707,166,"teacher: if we say a city is historic, a lot o...",60317fc2c8f6320ecde4bcb3,2
19965,65708,166,"teacher: yes, absolutely! is your hometown his...",60317fc2c8f6320ecde4bcb3,2
19966,65709,166,"teacher: i see! ok, lovely, i'll let you go no...",60317fc2c8f6320ecde4bcb3,2


In [4]:
# computing human-averaged values 
avg_vals = conv_data[['text','conversation_id','id','human_int']].groupby(['text','conversation_id','id',],
                                                                     ).agg(human_int_mean= ('human_int', 'mean'), 
                                                                           human_int_var= ('human_int', 'var'),
                                                                           ).reset_index()
avg_vals

,text,conversation_id,id,human_int_mean,human_int_var
0,student: teacher: does it snow a lot in ukra...,92,58405,1.333333,0.333333
1,student: teacher: has he been to the usa? i ...,92,58407,2.666667,0.333333
2,"student: teacher: ok <student>, excellent wo...",92,58410,2.333333,1.333333
3,student: ! sure! teacher: well done for learn...,131,35877,1.666667,1.333333
4,student: (*/*) i think i haven't teacher: som...,103,39257,1.666667,0.333333
...,...,...,...,...,...
6651,teacher: you've guessed the chamber of secrets...,103,65616,0.666667,0.333333
6652,teacher: your bookshop sounds like a lovely pl...,154,45838,1.333333,0.333333
6653,teacher: your classmates? you still don't have...,125,59667,1.333333,0.333333
6654,teacher: your noun 'production' is a good word...,20,64387,3.333333,0.333333


In [5]:
df1 = conv_data[['id','text','participant','human_int']].pivot(columns=['participant'], values='human_int', index='id').reset_index()
df1

participant,id,58c62eac444ade00010231a9,59b536d60516f6000197c3f9,5ab15ab4c42b630001cd36fb,5ad63c167f70c10001904bc5,5b830542b0d72600018dc280,5bb2a0148f3bd70001e5633f,5c4b06903566570001309394,5c4c747aa123930001ac124f,5c6ed5ea21a3560001bd3758,...,61139bc8479c2861f52c4a71,611a78efa6c36a674dc40cc6,611a9f9591ef82d19cfc5db1,611ccab657532350bc9b78ee,612a87998958706392797405,613d091096ca434d703f77c5,614e848825ae648282115670,61693c754827d0e35885a4ea,6171b1bc917dfe828a3130dd,6341258618bd26e47d901d74
0,22313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,22314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2,22315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,22316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
4,22317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6651,67110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6652,67111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6653,67112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6654,67113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
human_ratings_df = avg_vals.merge(df1, on='id')
human_ratings_df

,text,conversation_id,id,human_int_mean,human_int_var,58c62eac444ade00010231a9,59b536d60516f6000197c3f9,5ab15ab4c42b630001cd36fb,5ad63c167f70c10001904bc5,5b830542b0d72600018dc280,...,61139bc8479c2861f52c4a71,611a78efa6c36a674dc40cc6,611a9f9591ef82d19cfc5db1,611ccab657532350bc9b78ee,612a87998958706392797405,613d091096ca434d703f77c5,614e848825ae648282115670,61693c754827d0e35885a4ea,6171b1bc917dfe828a3130dd,6341258618bd26e47d901d74
0,student: teacher: does it snow a lot in ukra...,92,58405,1.333333,0.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,student: teacher: has he been to the usa? i ...,92,58407,2.666667,0.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"student: teacher: ok <student>, excellent wo...",92,58410,2.333333,1.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,student: ! sure! teacher: well done for learn...,131,35877,1.666667,1.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,student: (*/*) i think i haven't teacher: som...,103,39257,1.666667,0.333333,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6651,teacher: you've guessed the chamber of secrets...,103,65616,0.666667,0.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6652,teacher: your bookshop sounds like a lovely pl...,154,45838,1.333333,0.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6653,teacher: your classmates? you still don't have...,125,59667,1.333333,0.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
6654,teacher: your noun 'production' is a good word...,20,64387,3.333333,0.333333,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
human_ratings_df['human_int'] = human_ratings_df['human_int_mean'].apply(lambda x: round(x))

In [69]:
def find_int_gpt(rating_string):
    if isinstance(rating_string, str):
        if re.search(r'\d+', rating_string) :
            match = re.search(r'\b\d+\b', rating_string)
            return int(match[0])
    else:
        return "None"
    
    
def find_int_llama(llm_result):

    if re.search(r'INT:\s*(\d+)', llm_result) :
    # if 'INT' in llm_result:
        
        match = re.search(r'\b\d+\b', llm_result)
        return int(match[0])
    return 'None'


def squash_to_4(int_val):
    if int_val >4:
        int_val = 4
    return int_val


def find_int_value_nonchat(llm_result):

    if re.search(r'\d+', llm_result) :
    # if 'INT' in llm_result:
        
        match = re.search(r'\b\d+\b', llm_result)
        if int(match[0])>4:
            return 'None'
        return int(match[0])
    return 'None'

### GPT

In [9]:
gpt4_ratings = pd.read_excel("llm_responses/gpt4_total_responses.xlsx")
gpt4_ratings['gpt4_int'] = gpt4_ratings['rating'].apply(find_int_gpt)
gpt4_ratings

,proc_text,conversation_id,id,rating,gpt4_int
0,teacher: hello <student>! student: hi <teacher>!,118,22313,INT: 1\nEXP_INT: 3\nREASON: The conversation i...,1.0
1,teacher: hi - welcome to the chat how are you?...,118,22314,INT: 3 EXP_INT: 3 REASON: The teacher's welcom...,3.0
2,"teacher: oh austria, interesting! all the past...",118,22315,INT: 3\nEXP_INT: 3\nREASON: The teacher's comm...,3.0
3,"teacher: i'm from ukraine, but i've lived in t...",118,22316,INT: 3 \nEXP_INT: 3 \nREASON: This conversatio...,3.0
4,"teacher: yes, i was an english teacher and cam...",118,22317,INT: 4\nEXP_INT: 4\nREASON: The teacher's resp...,4.0
...,...,...,...,...,...
6651,"teacher: yes, good! what about equality'? it's...",167,66077,2: The student's reply shows they are thinking...,2.0
6652,"teacher: ok, i'll give you a hint two plus two...",167,66078,2: Moderately interesting.,2.0
6653,teacher: if men and women have the same rights...,167,66079,3: The student's response is pretty interestin...,3.0
6654,"teacher: they are not the same, but they are e...",167,66080,2 : This response is fairly neutral. The stude...,2.0


In [10]:
def get_int_gpt4o(response):
    try:
        json_resp = eval(response)
        return json_resp['INT']
    except:
        if re.search(r'"INT":\s*(\d+)', response) :
            match = re.search(r'\b\d+\b', response)
            return int(match[0])
        

In [11]:
gpt4o_ratings = pd.read_excel("llm_responses/gpt4o_ratings.xlsx")
gpt4o_ratings['gpt4o_int'] = gpt4o_ratings['gpt4o_responses'].apply(get_int_gpt4o)
gpt4o_ratings

,id,conversation_id,proc_text,gpt4o_responses,gpt4o_int
0,45939,7,"teacher: hi there <student>, all ok? student:...","{\n ""INT"": 1,\n ""EXP_INT"": 1,\n ""REASON"": ""...",1
1,45940,7,"teacher: yeah i'm good thanks, just been for a...","{\n ""INT"": 2,\n ""EXP_INT"": 2,\n ""REASON"": ""...",2
2,45941,7,teacher: yeah? what did you do? i'll come back...,"{\n ""INT"": 3,\n ""EXP_INT"": 3,\n ""REASON"": ""...",3
3,45942,7,teacher: ok i see ...too bad about the running...,"{\n ""INT"": 4,\n ""EXP_INT"": 4,\n ""REASON"": ""...",4
4,45943,7,teacher: yes i realise it's not easy really! i...,"{\n ""INT"": 2,\n ""EXP_INT"": 3,\n ""REASON"": ""...",2
...,...,...,...,...,...
6651,56386,258,teacher: but 'to loan' means 'to lend' studen...,"{\n ""INT"": 1,\n ""EXP_INT"": 1,\n ""REASON"": ""...",1
6652,56387,258,"teacher: banks loan money yes, exactly, it wou...","{\n ""INT"": 3,\n ""EXP_INT"": 3,\n ""REASON"": ""...",3
6653,56388,258,"teacher: ooh, we're out of time! student: or ...","{\n ""INT"": 2,\n ""EXP_INT"": 2,\n ""REASON"": ""...",2
6654,56389,258,"teacher: yes, absolutely! student: 13:59 on m...","{\n ""INT"": 1,\n ""EXP_INT"": 1,\n ""REASON"": ""...",1


In [12]:
all_gpt4_ratings = gpt4_ratings[['id','conversation_id','proc_text', 'gpt4_int']].merge(gpt4o_ratings[['id','conversation_id','proc_text', 'gpt4o_int']],
                                                                     on=['id','conversation_id','proc_text'], how='left')
all_gpt4_ratings.to_excel('data/all_gpt4_ratings.xlsx', index=False)

### LLAMA

In [13]:
new_7bchat = pd.read_json('llm_responses/llama2_chat7b_newprompt.json')
new_13bchat = pd.read_json('llm_responses/llama2_chat13b_newprompt.json')
print(len(new_13bchat), len(new_7bchat))
llama_data = pd.read_excel('data/conversations_for_llama_6656.xlsx')
llama_data['id'] = llama_data['id'].apply(int)
llama_data['conversation_id'] = llama_data['conversation_id'].apply(int)
llama_data['new_13bchat'] = new_13bchat[0]
llama_data['new_7bchat'] = new_7bchat[0]


6656 6656


In [14]:

with open('llm_responses/llama2_chat7b_oldprompt_initial.pkl', 'rb') as f:
    llama7bchat_ratings = pickle.load(f)
llama7bchat_ratings = ['INT: 1'] + llama7bchat_ratings
llama_missing_7bchat = pd.read_json('llm_responses/llama2_chat7b_oldprompt_missing.json')


with open("llm_responses/llama2_chat13b_oldprompt_2500.pkl", 'rb') as f:
    total_2500 = pickle.load(f)
with open("llm_responses/llama2_chat13b_oldprompt_2000to2500.pkl", 'rb') as f:
    bet_2000_2500 = pickle.load(f)
with open("llm_responses/llama2_chat13b_oldprompt_2000.pkl", 'rb') as f:
    upto_2000 = pickle.load(f)
llama13bchat_ratings = total_2500+bet_2000_2500+upto_2000
llama13bchat_ratings = ['INT: 1'] + llama13bchat_ratings[1:]
llama_missing_13bchat = pd.read_json('llm_responses/llama2_chat13b_oldprompt_missing.json')


llama_7bnonchat = pd.read_json('llm_responses/llama2_nonchat7b_newprompt.json')
llama_7bnonchat_missing = pd.read_json('llm_responses/llama2_nonchat7b_newprompt_missing.json')
llama_13bnonchat = pd.read_json('llm_responses/llama2_nonchat13b_newprompt.json')
llama_13bnonchat_missing = pd.read_json('llm_responses/llama2_nonchat13b_newprompt_missing.json')



In [15]:
conv_data1 = pd.read_excel("data/conversations_for_llama_6276.xlsx")
conv_data1["7bchat_llama"] = llama7bchat_ratings
conv_data1["13bchat_llama"] = llama13bchat_ratings



In [16]:
conv_data1['7b_int'] = conv_data1['7bchat_llama'].apply(lambda x: find_int_llama(x))
conv_data1['13b_int'] = conv_data1['13bchat_llama'].apply(lambda x: find_int_llama(x))

conv_data1['7b_int'] = conv_data1['7b_int'].apply(pd.to_numeric, errors='coerce')
conv_data1['13b_int'] = conv_data1['13b_int'].apply(pd.to_numeric, errors='coerce')

conv_data1['7b_int'] = conv_data1['7b_int'].apply(squash_to_4)
conv_data1['13b_int'] = conv_data1['13b_int'].apply(squash_to_4)

conv_data1['7bnonchat_int'] = llama_7bnonchat[0]
conv_data1['13bnonchat_int'] = llama_13bnonchat[0]

df1 = conv_data1[['id', 'conversation_id', '7b_int', '13b_int', '7bnonchat_int', '13bnonchat_int' ]]
df1 = df1[df1['id'].isin(llama_data['id'].unique())]

missing_conv_data = pd.read_excel("data/conversations_for_llama_missing.xlsx")
missing_conv_data['7b_int'] = llama_missing_7bchat
missing_conv_data['13b_int'] = llama_missing_13bchat
missing_conv_data['7bnonchat_int'] = llama_7bnonchat_missing[0]
missing_conv_data['13bnonchat_int'] = llama_13bnonchat_missing[0]
df2 = missing_conv_data[['id', 'conversation_id', '7b_int', '13b_int', '7bnonchat_int', '13bnonchat_int']]
df2 = df2[df2['id'].isin(llama_data['id'].unique())]
complete_llama_chat = pd.concat([df1, df2], ignore_index=True)
complete_llama_chat['13bnonchat_int'] = complete_llama_chat['13bnonchat_int'].apply(find_int_value_nonchat)
complete_llama_chat

,id,conversation_id,7b_int,13b_int,7bnonchat_int,13bnonchat_int
0,58899,77,4.0,4.0,3,2
1,58900,77,3.0,3.0,1,0
2,58901,77,4.0,4.0,0,2
3,58902,77,3.0,3.0,0,0
4,58903,77,4.0,4.0,1,2
...,...,...,...,...,...,...
6651,65706,166,2,2,0,2
6652,65707,166,3,3,1,2
6653,65708,166,2,2,0,2
6654,65709,166,1,3,0,1


In [70]:
llama_8bnonchat = pd.read_json('llm_responses/llama3_nonchat8b_newprompt.json')
llama_8bnonchat['llama3_nonchat_int'] = llama_8bnonchat['response'].apply(find_int_value_nonchat)
llama_8bnonchat

,id,text,prompt,response,llama3_nonchat_int
0,53285,"student: hi <teacher>, how are you? teacher: ...","The student said ""hi <teacher>, how are you? ...",2.\n\end{code}\n\nComment: Please provide enou...,2
1,45947,teacher: yeah well my daughter started doing i...,"The student said ""oh no, it's a mistake. i tho...",3\n\nIn order to know if it was a mistake or n...,3
2,45955,teacher: yeah? in the uk too i'd say....there ...,"The student said ""i think it's great! i can se...","2\n The student said ""it's nice to see more pe...",2
3,45941,teacher: yeah? what did you do? i'll come back...,"The student said ""every morning i try to do so...",1\n To decide if the teacher wants to talk wit...,1
4,53297,student: great teacher: but not 'as a team' j...,"The student said ""great teacher: but not 'as ...",3 to the previous example.\n\nTeacher: I will ...,3
...,...,...,...,...,...
6651,38329,"student: but then, if you have some exercises ...","The student said ""but then, if you have some e...",0 for the first message and 1 for the second o...,0
6652,56381,teacher: my guess is that in italian 'borrow' ...,"The student said ""if barbara's father had grew...",3.\n To evaluate if the message contains infor...,3
6653,56382,teacher: nearly! had ___ up in another time s...,"The student said ""they are indeed, but to be f...","2.\n\nThe teacher asked ""what is your question...",2
6654,38323,student: maybe i should start by writing a top...,"The student said ""maybe i should start by writ...",2.\n\n## Topic sentence\n\n A topic sentence o...,2


In [ ]:
nonchat_ids = llama_8bnonchat['id'].unique()

In [43]:
llama_8bchat = pd.read_json('llm_responses/llama3_chat8b.json')
ids = list(llama_8bchat['id'].unique())

In [44]:
llama_8bchat1 = pd.read_json('llm_responses/llama3_chat8b_responses_alconv.json')
ids1 = list(llama_8bchat1['id'].unique())

In [76]:
total_llama_8bchat = pd.concat([llama_8bchat, llama_8bchat1])
total_llama_8bchat['llama3_chat_int'] = total_llama_8bchat['response'].apply(find_int_llama).apply(squash_to_4)
total_llama_8bchat

,id,text,prompt,response,llama3_chat_int
0,59738,student: good afternoon <teacher> student: ...,You are an AI evaluator rating how interesting...,INT: 3; EXP_INT: 4; REASON: The student's resp...,3
1,59739,teacher: hi <student> - how are you? student:...,You are an AI evaluator rating how interesting...,INT: 4; EXP_INT: 3; REASON: The student's resp...,4
2,59740,teacher: yes i guess you must be used to it by...,You are an AI evaluator rating how interesting...,INT: 3; EXP_INT: 3; REASON: The student's resp...,3
3,59741,teacher: right i see...yes i'd really like to ...,You are an AI evaluator rating how interesting...,INT: 1; EXP_INT: 1; REASON: The student's resp...,1
4,59742,"teacher: yes,,you'll look back on it with nost...",You are an AI evaluator rating how interesting...,INT: 0; EXP_INT: 0; REASON: The student's resp...,0
...,...,...,...,...,...
5897,67110,student: let me think past simple! teacher: o...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 3;\nEXP_INT: 3;\nREA...,3
5898,67111,student: because in recent year is present per...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 4;\nEXP_INT: 3;\nREA...,4
5899,67112,student: yes.. i think i can understand it now...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 1;\nEXP_INT: 2;\nREA...,1
5900,67113,student: yes that is what i like to do teache...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 0;\nEXP_INT: 1;\nREA...,0


In [78]:
total_llama_8bchat = total_llama_8bchat[total_llama_8bchat['id'].isin(complete_llama_chat['id'].unique())]
total_llama_8bchat

,id,text,prompt,response,llama3_chat_int
0,59738,student: good afternoon <teacher> student: ...,You are an AI evaluator rating how interesting...,INT: 3; EXP_INT: 4; REASON: The student's resp...,3
1,59739,teacher: hi <student> - how are you? student:...,You are an AI evaluator rating how interesting...,INT: 4; EXP_INT: 3; REASON: The student's resp...,4
2,59740,teacher: yes i guess you must be used to it by...,You are an AI evaluator rating how interesting...,INT: 3; EXP_INT: 3; REASON: The student's resp...,3
3,59741,teacher: right i see...yes i'd really like to ...,You are an AI evaluator rating how interesting...,INT: 1; EXP_INT: 1; REASON: The student's resp...,1
4,59742,"teacher: yes,,you'll look back on it with nost...",You are an AI evaluator rating how interesting...,INT: 0; EXP_INT: 0; REASON: The student's resp...,0
...,...,...,...,...,...
5897,67110,student: let me think past simple! teacher: o...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 3;\nEXP_INT: 3;\nREA...,3
5898,67111,student: because in recent year is present per...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 4;\nEXP_INT: 3;\nREA...,4
5899,67112,student: yes.. i think i can understand it now...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 1;\nEXP_INT: 2;\nREA...,1
5900,67113,student: yes that is what i like to do teache...,You are an AI evaluator rating how interesting...,Here's my rating:\n\nINT: 0;\nEXP_INT: 1;\nREA...,0


In [80]:
complete_llama_chat.merge(total_llama_8bchat[['id','llama3_chat_int']], on='id',how='left').merge(llama_8bnonchat[['id','llama3_nonchat_int']], on='id',how='left')

,id,conversation_id,7b_int,13b_int,7bnonchat_int,13bnonchat_int,llama3_chat_int,llama3_nonchat_int
0,58899,77,4.0,4.0,3,2,3,3
1,58900,77,3.0,3.0,1,0,2,2
2,58901,77,4.0,4.0,0,2,1,0
3,58902,77,3.0,3.0,0,0,0,0
4,58903,77,4.0,4.0,1,2,3,0
...,...,...,...,...,...,...,...,...
6651,65706,166,2,2,0,2,3,1
6652,65707,166,3,3,1,2,2,2
6653,65708,166,2,2,0,2,3,0
6654,65709,166,1,3,0,1,1,2


In [81]:
complete_llama_chat.to_excel('data/all_llama_ratings.xlsx', index=False)

### MISTRAL

In [42]:
mistral_7b = pd.read_excel('/Users/mahathi/Dissertation/total_ratings/mistral7b_complete.xlsx')
## need to add missing values
mistral_7b_missing = pd.read_json("/Users/mahathi/Downloads/mistral7b_ins_score_list_final.json")

with open("/Users/mahathi/Dissertation/total_ratings/mix7x8_1000.pkl", 'rb') as f:
    mixtral_1 = pickle.load(f)

with open("/Users/mahathi/Dissertation/total_ratings/mixtral_all1.pkl", 'rb') as f:
    mixtral_2 = pickle.load(f)

In [43]:
mistral_7b = mistral_7b[mistral_7b['id'].isin(human_ratings_df['id'].unique())]
mistral_7b = mistral_7b.rename(columns={'proc_text':'text'})
missing_conv_data_mis = pd.read_excel("/Users/mahathi/Downloads/missing_conv_data_toadd.xlsx")

missing_conv_data_mis['mis7_int'] = mistral_7b_missing[0]
missing_conv_data_mis = missing_conv_data_mis[missing_conv_data_mis['id'].isin(human_ratings_df['id'].unique())]
missing_conv_data_mis
complete_mis_ins = pd.concat([mistral_7b[['text','id','conversation_id','mis7_int']], missing_conv_data_mis], ignore_index=True)
mistral_nonchat = pd.read_json('/Users/mahathi/Downloads/mistral7b_score_list_final.json')
mistral_data = pd.read_excel('/Users/mahathi/Dissertation/LLM_Analysis/FINAL_new_prompts.xlsx')
mistral_data['id'] = mistral_data['id'].apply(int)
mistral_data['conversation_id'] = mistral_data['conversation_id'].apply(int)
mistral_data['mis_nonchat_int'] = mistral_nonchat[0]
mistral_data['mixtral'] = mixtral_1 + mixtral_2
mistral_data['mix_int'] = mistral_data['mixtral'].apply(find_int_llama)
mistral_data_final = mistral_data.merge(complete_mis_ins, on=['conversation_id', 'id'], how='left')

In [54]:
mistral_data_final[['id', 'conversation_id', 'proc_text', 'mis_nonchat_int', 'mix_int', 'mis7_int']].sort_values(['conversation_id', 'id']).to_excel('data/all_mistral_ratings.xlsx', index=False)